In [4]:
# Importando as bibliotecas necessárias
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  # Para pré-processamento de dados
from sklearn.utils import shuffle  # Para embaralhar os dados
from sklearn.model_selection import train_test_split  # Para dividir o conjunto de dados em treinamento e teste
from sklearn.metrics import (  # Métricas de avaliação de modelo
    confusion_matrix, accuracy_score, recall_score, f1_score, roc_auc_score, classification_report
)

from tensorflow.keras.optimizers import Adam  # Otimizador para a rede neural
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Pré-processamento de imagens

import keras  # Biblioteca Keras para criação de modelos de redes neurais
import tensorflow as tf  # TensorFlow para construção de redes neurais

import matplotlib.pyplot as plt  # Para plotagem de gráficos
import pandas as pd  # Manipulação de dados em formato tabular
import numpy as np  # Biblioteca NumPy para manipulação de matrizes
import cv2  # OpenCV para processamento de imagens
import os  # Operações com sistema de arquivos


In [5]:
path = 'Dataset/'  # Definindo o caminho do diretório contendo o dataset
im_size = 350  # Definindo o tamanho desejado para as imagens
path_test = 'Dataset_Test/'  # Path to the test dataset

street_types = os.listdir(path)  # Listando os tipos de ruas disponíveis no diretório
street_types_test = os.listdir(path_test)

print(street_types)  # Imprimindo os tipos de ruas encontrados
print("Tipos de ruas encontrados: ", len(street_types))  # Imprimindo o número de tipos de ruas encontrados

['clean', 'litter', 'recycle']
Tipos de ruas encontrados:  3


In [6]:
streets = []  # Lista vazia para armazenar informações das ruas

# Iterando sobre os diferentes tipos de ruas
for item in street_types:
    all_streets = os.listdir(path + '/' + item)  # Obtendo todos os nomes de arquivo na pasta
    
    # Adicionando os nomes de arquivo à lista 'streets'
    for street in all_streets:
        streets.append((item, str(path + '/' + item) + '/' + street))

# Criando um DataFrame do pandas com as informações das ruas
streets_df = pd.DataFrame(data=streets, columns=['street type', 'image'])

In [7]:
# Imprimindo o número total de ruas no conjunto de dados
print("Total number of streets in the Dataset: ", len(streets_df))

# Contando o número de ruas em cada categoria e imprimindo os resultados
street_count = streets_df['street type'].value_counts()
print("Ruas em cada categoria: ")
print(street_count)

Total number of streets in the Dataset:  4805
Ruas em cada categoria: 
recycle    1675
clean      1625
litter     1505
Name: street type, dtype: int64


In [8]:
# Lista vazia para armazenar informações das ruas para teste
streets_test = []

# Iterando sobre os mesmos tipos de ruas, agora no conjunto de testes
for item in street_types:
    all_streets_test = os.listdir(os.path.join(path_test, item))  # Obtendo todos os nomes de arquivo na pasta de teste
    
    # Adicionando os nomes de arquivo à lista 'streets_test'
    for street in all_streets_test:
        streets_test.append((item, os.path.join(path_test, item, street)))

# Criando um DataFrame do pandas com as informações das ruas para teste
streets_test_df = pd.DataFrame(data=streets_test, columns=['street type', 'image'])

In [9]:
# Imprimindo o número total de ruas no conjunto de dados
print("Total number of streets in the Dataset_test: ", len(streets_test_df))

# Contando o número de ruas em cada categoria e imprimindo os resultados
street_test_count = streets_test_df['street type'].value_counts()
print("Ruas em cada categoria: ")
print(street_test_count)

Total number of streets in the Dataset_test:  600
Ruas em cada categoria: 
clean      200
litter     200
recycle    200
Name: street type, dtype: int64


In [10]:
images = []  # Lista para armazenar as imagens
labels = []  # Lista para armazenar as etiquetas das ruas

# Iterando sobre os diferentes tipos de ruas
for label in street_types:
    data_path = os.path.join(path, label)  # Obtendo o caminho para os dados de uma categoria
    filenames = os.listdir(data_path)  # Obtendo os nomes dos arquivos de imagem na categoria

    # Iterando sobre os arquivos de imagem na categoria
    for filename in filenames:
        img_path = os.path.join(data_path, filename)  # Obtendo o caminho completo da imagem
        img = load_img(img_path, target_size=(im_size, im_size))  # Carregando a imagem e redimensionando
        img_array = img_to_array(img)  # Convertendo a imagem para um array numpy
        images.append(img_array)  # Adicionando a imagem à lista de imagens
        labels.append(label)  # Adicionando a etiqueta da rua à lista de etiquetas

# Convertendo a lista de imagens e etiquetas para arrays numpy
images = np.array(images)
labels = np.array(labels)

# Normalizando os valores de pixel
images = images.astype('float32') / 255.0

# Verificando a forma (shape) dos arrays
print("Shape das imagens:", images.shape)
print("Shape das etiquetas:", labels.shape)

Shape das imagens: (4805, 350, 350, 3)
Shape das etiquetas: (4805,)


In [11]:
# Processamento das imagens de teste
images_test = []  # Lista para armazenar as imagens de teste
labels_test = []  # Lista para armazenar as etiquetas das ruas de teste

# Iterando sobre os diferentes tipos de ruas para teste
for label in street_types:
    data_path_test = os.path.join(path_test, label)  # Obtendo o caminho para os dados de uma categoria de teste
    filenames_test = os.listdir(data_path_test)  # Obtendo os nomes dos arquivos de imagem na categoria de teste

    # Iterando sobre os arquivos de imagem na categoria de teste
    for filename in filenames_test:
        img_path = os.path.join(data_path_test, filename)  # Obtendo o caminho completo da imagem de teste
        img = load_img(img_path, target_size=(im_size, im_size))  # Carregando a imagem de teste e redimensionando
        img_array = img_to_array(img)  # Convertendo a imagem de teste para um array numpy
        images_test.append(img_array)  # Adicionando a imagem de teste à lista de imagens de teste
        labels_test.append(label)  # Adicionando a etiqueta da rua de teste à lista de etiquetas de teste

# Convertendo a lista de imagens e etiquetas para arrays numpy para teste
images_test = np.array(images_test)
labels_test = np.array(labels_test)

# Normalizando os valores de pixel para teste
images_test = images_test.astype('float32') / 255.0

# Verificando a forma (shape) dos arrays para teste
print("Shape das imagens (teste):", images_test.shape)
print("Shape das etiquetas (teste):", labels_test.shape)

Shape das imagens (teste): (600, 350, 350, 3)
Shape das etiquetas (teste): (600,)


In [12]:
# Obtendo as etiquetas das ruas do DataFrame de treino e validação
y_train = streets_df['street type'].values

# Convertendo as etiquetas para valores numéricos usando LabelEncoder
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Embaralhando as imagens de treino e suas etiquetas para garantir uma mistura aleatória
train_images, train_labels = shuffle(images, y_train_encoded, random_state=1)

# Dividindo os dados apenas em conjuntos de treinamento e validação (não mais teste aqui)
train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels, test_size=0.05, random_state=415)

# Imprimindo as formas (shapes) dos conjuntos de dados de treino e validação
print("Train shape:", train_x.shape)
print("Validation shape:", val_x.shape)
print("Train labels shape:", train_y.shape)
print("Validation labels shape:", val_y.shape)

# Agora, para o conjunto de teste
# Primeiro, extrai as etiquetas do DataFrame de teste
y_test = streets_test_df['street type'].values
# Aqui, você transforma as etiquetas de teste usando o mesmo label encoder usado para o treinamento
# Isto é importante para garantir que as etiquetas correspondam entre os conjuntos de treino e teste
test_labels = label_encoder.transform(y_test)

# Seu conjunto de teste já foi carregado e processado nas etapas anteriores, então aqui você só precisa verificar seus shapes
print("Test shape:", images_test.shape)
print("Test labels shape:", test_labels.shape)

Train shape: (4564, 350, 350, 3)
Validation shape: (241, 350, 350, 3)
Train labels shape: (4564,)
Validation labels shape: (241,)
Test shape: (600, 350, 350, 3)
Test labels shape: (600,)


In [14]:
# Calculando a matriz de confusão para o conjunto de teste
confusion_test = confusion_matrix(test_labels, y_pred_test)
print("Matriz de Confusão (Teste):")
print(confusion_test)

# Gerando um relatório de classificação para o conjunto de teste
report_test = classification_report(test_labels, y_pred_test, target_names=label_encoder.classes_, zero_division=1)
print("Relatório de Classificação (Teste):")
print(report_test)

# Calculando a Área sob a Curva (AUC) para cada classe no conjunto de teste
for i in range(len(label_encoder.classes_)):
    auc = roc_auc_score(test_labels == i, y_probs_test[:, i])
    print(f"AUC para classe {label_encoder.classes_[i]}: {auc}")

Matriz de Confusão (Teste):
[[163  29   8]
 [  2 198   0]
 [ 11   6 183]]
Relatório de Classificação (Teste):
              precision    recall  f1-score   support

       clean       0.93      0.81      0.87       200
      litter       0.85      0.99      0.91       200
     recycle       0.96      0.92      0.94       200

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600

AUC para classe clean: 0.96539375
AUC para classe litter: 0.98608125
AUC para classe recycle: 0.991225
